In [270]:
import numpy as np
import pandas as pd
from sklearn import metrics
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import f_oneway, tukey_hsd, ttest_1samp

In [298]:
dataset = "Liver"
datatype = "log2" # "raw_counts"
clustertype = "kmeans" # "kmeans"
plottype = clustertype
if clustertype != 'kmeans':
    plottype = "top1"

datafolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\data\\"
resfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\results\\"+dataset+"\\"+datatype+"\\"
plotfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\plots\\"+dataset+"\\"+datatype+"\\"+plottype+"\\"+"classification\\"
scorenames = ["z", "gsva", "auc", "cerno",
              "aucell", "vision", "ratios",
              "svd", "sparse_pca"]
plot_scorenames = {"AUCell": "aucell",
                   "CERNO": "auc",
                   "CERNO F": "cerno",
                   "Vision": "vision",
                   "z-score": "z",
                   "DropRatio": "ratios",
                   "PLAGE": "svd",
                   "sparsPCA": "sparse_pca",
                   "GSVA": "gsva"}

In [3]:
names = ["Balanced_accuracy_",
        "ARI_",
        "F1_",
        "Recall_",
        "Matthews_",
        "Jaccard_",
        "Hamming_",
        "Precision_",
        "FDR_"]

## Get chosen pathways for cell types

In [4]:
paths = pd.read_csv(datafolder+"chosen_paths.txt", sep='\t', index_col=0)
paths.head()

,ID,Title,Category,Database,GSsize,GenesInGS,Perc,Celltype,Celltype_unclear
hsa04062,hsa04062,KEGG: Chemokine signaling pathway,KEGG: Organismal Systems; Immune system,KEGG,192,139,72.395833,T cell,no
hsa04610,hsa04610,KEGG: Complement and coagulation cascades,KEGG: Organismal Systems; Immune system,KEGG,85,46,54.117647,B cell; T cell,no
hsa04612,hsa04612,KEGG: Antigen processing and presentation,KEGG: Organismal Systems; Immune system,KEGG,78,66,84.615385,NK cell; T cell,no
hsa04650,hsa04650,KEGG: Natural killer cell mediated cytotoxicity,KEGG: Organismal Systems; Immune system,KEGG,131,94,71.755725,NK cell,no
hsa04657,hsa04657,KEGG: IL-17 signaling pathway,KEGG: Organismal Systems; Immune system,KEGG,94,63,67.021277,NK cell; T cell,no


## Get dataset specific pathways

In [4]:
geneset_info = pd.read_csv(datafolder + dataset +"//genesets_modules.csv", index_col=0)
geneset_info.head()

,ID,Title,source,DataBase
hsa04062,hsa04062,Chemokine signaling pathway,Organismal Systems; Immune system,KEGG
hsa04610,hsa04610,Complement and coagulation cascades,Organismal Systems; Immune system,KEGG
hsa04611,hsa04611,Platelet activation,Organismal Systems; Immune system,KEGG
hsa04612,hsa04612,Antigen processing and presentation,Organismal Systems; Immune system,KEGG
hsa04613,hsa04613,Neutrophil extracellular trap formation,Organismal Systems; Immune system,KEGG


## Choose the ones in both

In [6]:
dataset_specific = paths[paths["ID"].isin(geneset_info["ID"])]
dataset_specific.Celltype = dataset_specific['Celltype'].str.replace(';',' +')

C:\Users\amruk\AppData\Local\Temp\ipykernel_20512\2728621780.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_specific.Celltype = dataset_specific['Celltype'].str.replace(';',' +')


In [7]:
to_save = dataset_specific[["ID", "Title", "Celltype"]]

## Get true labels

In [10]:
true_labels = pd.read_csv(datafolder + dataset +"//true_labels.csv", index_col=0)
true_labels.head()

,ID,Cell.type.authors
2,SAMEA11294524-AAACCTGAGTCTCCTC,hepatocyte
3,SAMEA11294524-AAACCTGCAATGTAAG,monocyte-derived macrophage
4,SAMEA11294524-AAACCTGCACCGATAT,T cell
6,SAMEA11294524-AAACCTGCAGTCAGAG,liver sinusoidal endothelial cell
7,SAMEA11294524-AAACCTGGTATCAGTC,T cell


In [ ]:
true_labels.set_index('NAME', inplace=True)

In [11]:
true_labels.set_index('ID', inplace=True)

In [12]:
true_labels = true_labels.rename(columns={'Cell.type.Ontology': 'CellType'})
true_labels = true_labels.rename(columns={'Cell.type.authors': 'CellType'})
true_labels.CellType.unique()

array(['hepatocyte', 'monocyte-derived macrophage', 'T cell',
       'liver sinusoidal endothelial cell', 'natural killer cell',
       'cycling cell', 'plasma cell', 'B cell', 'cholangiocyte',
       'Kupffer cell', 'vascular endothelial cell 1',
       'vascular endothelial cell 2', 'activated HSC', 'quiescent HSC'],
      dtype=object)

In [13]:
not_pre_B = ~true_labels["CellType"].isin(['precursor B cell',
                                                         'pro-B cell'])
true_labels = true_labels[not_pre_B]

In [14]:
true_labels.loc[true_labels["CellType"].isin(['CD4+ T cell',
                                              'Cytotoxic T cell']), "CellType"] = 'T cell'
true_labels.loc[true_labels["CellType"].isin(['mature B cell']), "CellType"] = 'B cell'
true_labels.loc[true_labels["CellType"].isin(['Natural killer cell', 'natural killer cell']), "CellType"] = 'NK cell'
true_labels.loc[~true_labels["CellType"].isin(['NK cell',
                                               'T cell',
                                               'B cell']), "CellType"] = 'other'
true_labels.CellType.unique()

array(['other', 'T cell', 'NK cell', 'B cell'], dtype=object)

## Calculate classification scores

In [15]:
class Scores:
    def __init__(self):
        self.scoring_methods = [metrics.balanced_accuracy_score,
                                metrics.adjusted_rand_score,
                                metrics.f1_score,
                                metrics.recall_score,
                                metrics.matthews_corrcoef,
                                metrics.jaccard_score,
                                metrics.hamming_loss,
                                metrics.precision_score
                                ]
        self.names = names
        self.scores = [[] for _ in self.names]

    def get_classification_scores(self, y_true, y_pred):
        for i, scoring in enumerate(self.scoring_methods):
            res = scoring(y_true, y_pred)
            self.scores[i].append(res)
        self.scores[-1].append(1-res)

    def save_confusion_matrix(self, y_true, y_pred, resfolder, scorename, gs_name):
        mx = metrics.confusion_matrix(y_true, y_pred)
        df = pd.DataFrame(mx)
        df.index.name = 'true label'
        df.columns.name = 'predicted label'
        df.to_csv(resfolder+"confusion_matrix_"+plottype+"\\"+gs_name+"_"+scorename+".csv")

In [16]:
to_save = to_save[to_save.Celltype.str.contains('|'.join(true_labels.CellType.unique()))]

In [17]:
for scorename in list(plot_scorenames.values()):
    scores = pd.read_csv(resfolder+ scorename + ".csv", index_col=0)
    scores = scores.loc[:, not_pre_B]
    thresholds = pd.read_csv(
        resfolder+ scorename +"_" + clustertype + "_thr" ".csv",
        index_col=0)
    eval = Scores()
    for index, row in to_save.iterrows():
        gs_score = scores.loc[row["ID"]]
        thr = thresholds.loc[row["ID"]].max()
        preds = gs_score > thr
        preds = preds.astype(int)
        true_labels["label"] = true_labels.CellType.isin(
            row["Celltype"].split(" + ")
            ).astype(int)
        eval.get_classification_scores(true_labels["label"], preds)
        eval.save_confusion_matrix(true_labels["label"], preds,
                                   resfolder, scorename, row["ID"])
    for i, cls_score in enumerate(eval.scores):
        to_save.loc[:, eval.names[i]+scorename] = cls_score

to_save.to_csv(resfolder+"classification_scores_"+plottype+".csv")

c:\Users\amruk\Miniconda3\envs\enrich\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amruk\Miniconda3\envs\enrich\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amruk\Miniconda3\envs\enrich\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\amruk\Miniconda3\envs\enrich\lib\site-packages\sklearn\metrics\_classi

## Plots

In [296]:
def check_differences(df, name):
    pval = f_oneway(*[list(df[name+sc_name]) for sc_name in list(plot_scorenames.values())]).pvalue
    subtitle = "<br>ANOVA p-value : {}".format(np.round(pval, 3))
    pvals = np.ones((len(list(plot_scorenames.values())), len(list(plot_scorenames.values()))))
    if pval < 0.05:
        pvals = tukey_hsd(*[list(df[name+sc_name]) for sc_name in list(plot_scorenames.values())]).pvalue
        if pval < 0.001:
            subtitle = "<br>ANOVA p-value < 0.001"
    return pvals, subtitle


def get_brackets(pvals):
    dtype = [("start_idx", int), ("end_idx", int), ("dist", int), ("pvalue", "S10")]
    brackets = np.empty(0, dtype=dtype)
    for i in range(pvals.shape[0]):
        for j in range(i+1, pvals.shape[1]):
            if pvals[i, j] <= 0.05:
                text = str(np.round(pvals[i, j], 3))
                if pvals[i, j] < 0.001:
                    text = "<0.001"
                bracket = (i,
                           j,
                           j-i,
                           text)
                bracket = np.array(bracket, dtype=dtype)
                brackets = np.append(brackets, bracket)
    brackets = np.sort(brackets, order=["dist", "start_idx"])
    return brackets


def add_brackets(brackets, fig):
    lower = 1.0
    upper = 1.05
    for bracket in brackets:
        i = bracket["start_idx"]
        j = bracket["end_idx"]
        x = [list(plot_scorenames.keys())[i]] + list(plot_scorenames.keys())[i:j] + [list(plot_scorenames.keys())[j]]*2
        y = [lower]+ [upper]*(j-i+1)+[lower]
        fig.add_trace(go.Scatter(x=x,
                                 y=y,
                                 fill=None,
                                 mode="lines",
                                 line=dict(color='rgba(0,0,0,1)', width=1),
                                 showlegend=False
                                 ))
        fig.add_annotation(text=str(bracket["pvalue"].decode('UTF-8')),
                           name="p-value",                                  
                           x=(j+i)/2,
                           y=upper+0.04,
                           showarrow=False,
                           font=dict(size=10, color="black")
                           )
        lower = lower + 0.1
        upper = upper + 0.1
    return fig


def mark_different_boxes(fig, pvals):
    brackets = get_brackets(pvals)
    fig = add_brackets(brackets, fig)
    return fig


def visualize(df, cell_types, namescores, plot_folder):
    for celltype in cell_types:
        for name in namescores:
            vis = df.loc[df["Celltype"].isin([celltype]), df.columns.str.contains(name)]
            pvals, subtitle = check_differences(vis, name)
            vis.columns = vis.columns.str.replace(name, "")
            vis = vis.melt()
            vis = vis.rename(columns={'variable': 'method'})
            vis["method"] = vis["method"].map({v: k for k, v in plot_scorenames.items()})
            fig = px.box(vis, x="method", y="value", color="method",
                         title=celltype+subtitle,
                         template="plotly_white",
                         labels={"method": "PAS method", "value": name[:-1].replace("_", " ")},
                         category_orders={"method": list(plot_scorenames.keys())})
            fig = mark_different_boxes(fig, pvals)
            fig.update_xaxes(tickangle=45)
            fig.write_image(plot_folder + celltype.replace(" ", "_") + "_" + name[:-1] + ".png")


In [359]:
def visualize_difference(df1, df2, namescores, plot_folder):
    df = df1.subtract(df2)
    for name in namescores:
        vis = df.loc[:, df.columns.str.contains(name)]
        vis.columns = vis.columns.str.replace(name, "")
        vis = vis.melt()
        vis = vis.rename(columns={'variable': 'method'})
        vis["method"] = vis["method"].map({v: k for k, v in plot_scorenames.items()})
        fig = px.box(vis, x="method", y="value", color="method",
                        title=name[:-1].replace("_", " "),
                        template="plotly_white",
                        labels={"method": "PAS method", "value": "Difference [Top1 - k-means]"},
                        category_orders={"method": list(plot_scorenames.keys())})
        fig.add_hline(y=0.0, line=dict(dash='dash', color = "firebrick", width = 1))
        fig.update_xaxes(tickangle=45)
        for score in list(plot_scorenames.keys()):
            vals = vis.loc[vis["method"]==score]
            pval = ttest_1samp(vals["value"], 0.0).pvalue
            if pval < 0.05:
                text = str(np.round(pval, 3))
                if pval < 0.001:
                    text = "<0.001"
                y = vals["value"].min()-0.05
                if vals["value"].mean() > 0:
                    y = vals["value"].max()+0.05
                fig.add_annotation(text=text,
                        name="p-value",                                  
                        x=score,
                        y=y,
                        showarrow=False,
                        font=dict(size=10, color="red")
                        )
        fig.add_annotation(text="Better Top1",
                           name="p-value",                                  
                           x=0.5,
                           y=vis["value"].max()+0.15,
                           xref="paper",
                           showarrow=False,
                           font=dict(size=12, color="black")
                           )
        fig.add_annotation(text="Better k-means",
                           name="p-value",                                  
                           x=0.5,
                           y=vis["value"].min()-0.15,
                           xref="paper",
                           showarrow=False,
                           font=dict(size=12, color="black")
                           )
        fig.write_image(plot_folder + "difference_" + name[:-1] + ".png")

### COVID

In [299]:
plotfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\plots\\COVID\\"+datatype+"\\"+plottype+"\\"+"classification\\"
resfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\results\\COVID\\"+datatype+"\\"
covid_df = pd.read_csv(resfolder+"classification_scores_"+plottype+".csv", index_col=0)
groups = covid_df.groupby(["Celltype"])["Celltype"].count()
groups

Celltype
B cell              10
B cell + T cell      1
NK cell + T cell     2
T cell              14
Name: Celltype, dtype: int64

In [297]:
celltypes = groups[groups > 2].keys().tolist()
visualize(covid_df, celltypes, names, plotfolder)

In [360]:
plotfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\plots\\COVID\\"+datatype+"\\"
resfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\results\\COVID\\"+datatype+"\\"
covid_kmeans_df = pd.read_csv(resfolder+"classification_scores_kmeans.csv", index_col=0)
covid_top1_df = pd.read_csv(resfolder+"classification_scores_top1.csv", index_col=0)

cols = ["ID", "Title", "Celltype"]
covid_top1_df.drop(columns=cols, inplace=True)
covid_kmeans_df.drop(columns=cols, inplace=True)
visualize_difference(covid_top1_df, covid_kmeans_df, names, plotfolder)

### BM

In [12]:
plotfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\plots\\BM\\"+datatype+"\\"+plottype+"\\"+"classification\\"
resfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\results\\BM\\"+datatype+"\\"
bm_df = pd.read_csv(resfolder+"classification_scores_"+plottype+".csv", index_col=0)
groups = bm_df.groupby(["Celltype"])["Celltype"].count()
groups

Celltype
B cell              9
B cell + T cell     1
NK cell             6
NK cell + T cell    3
Name: Celltype, dtype: int64

In [ ]:
celltypes = groups[groups > 2].keys().tolist()
visualize(bm_df, celltypes, names, plotfolder)

### PBMC

In [14]:
plotfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\plots\\PBMC\\"+datatype+"\\"+plottype+"\\"+"classification\\"
resfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\results\\PBMC\\"+datatype+"\\"
pbmc_df = pd.read_csv(resfolder+"classification_scores_"+plottype+".csv", index_col=0)
groups = pbmc_df.groupby(["Celltype"])["Celltype"].count()
groups

Celltype
B cell              10
B cell + T cell      1
NK cell              7
NK cell + T cell     4
T cell              14
Name: Celltype, dtype: int64

In [ ]:
celltypes = groups[groups > 2].keys().tolist()
visualize(pbmc_df, celltypes, names, plotfolder)

### Liver

In [16]:
plotfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\plots\\Liver\\"+datatype+"\\"+plottype+"\\"+"classification\\"
resfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\results\\Liver\\"+datatype+"\\"
liver_df = pd.read_csv(resfolder+"classification_scores_"+plottype+".csv", index_col=0)
groups = liver_df.groupby(["Celltype"])["Celltype"].count()
groups

Celltype
B cell              10
B cell + T cell      1
NK cell              6
NK cell + T cell     4
T cell              12
Name: Celltype, dtype: int64

In [17]:
celltypes = groups[groups > 2].keys().tolist()
visualize(liver_df, celltypes, names, plotfolder)

### Merged

In [18]:
plotfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\plots\\merged\\"+datatype+"\\"+plottype+"\\"

merged = pd.concat([covid_df, pbmc_df, liver_df, bm_df])
groups = merged.groupby(["Celltype"])["Celltype"].count()
groups

Celltype
B cell              39
B cell + T cell      4
NK cell             19
NK cell + T cell    13
T cell              40
Name: Celltype, dtype: int64

In [19]:
celltypes = groups[groups > 2].keys().tolist()
visualize(merged, celltypes, names, plotfolder)